# Finding the needle in the haystack with the XCS algorithm

In this lab session, we will use the XCS algorithm to find the famous "needle in the haystack". In our case, the haystack will be an array of a certain size, with the needle positioned at a specific location. Our algorithm must be able to learn a set of rules to find a single important input bit (the "needle") from among a large array of irrelevant input bits (the "haystack").
We will use the ```xcs``` Python library that, obviously, implements the XCS algorithm. Let us import some useful modules and utilities.

In [8]:
import xcs
import random
from xcs.scenarios import Scenario
from xcs.bitstrings import BitString

We need to define a custom Scenario for our haystack problem. We can inherit from the Scenario abstract class defined in the ```xcs.scenarios``` submodule. Specifically, you need to define the following methods and properties:

- ```__init__```, taking 2 parameters: ```training_cycles```, to determine how many reward cycles the algorithm has to identify the "needle", and another parameter, ```input_size```, to determine how big the "haystack" is
- ```is_dynamic```, a property with a Boolean value that indicates whether the actions from one reward cycle can affect the rewards or situations of later reward cycles.
- ```get_possible_actions```, a method returning the actions the algorithm can take.
- ```reset```, a method performing the actions to restart the problem for a new run.
- ```sense```, a method which returns a new input (the "situation").
- ```execute```, a method which takes an action from among those returned by get_possible_actions(), in response to the last situation that was returned by sense(). It then returns a reward value indicating how well the algorithm is doing at responding correctly to each situation.
- ```more```, a methods which returns a Boolean value to indicate whether the algorithm has remaining reward cycles in which to learn.

In [9]:
class HaystackScenario(Scenario):
    
    def __init__(self, training_cycles=1000, input_size=500):
        self.input_size = input_size
        self.possible_actions = (True, False)
        self.initial_training_cycles = training_cycles
        self.remaining_cycles = training_cycles
        self.needle_index = random.randrange(input_size)
        self.needle_value = None

    @property
    def is_dynamic(self):
        return False
        
    def get_possible_actions(self):
        return self.possible_actions
    
    def reset(self):
        self.remaining_cycles = self.initial_training_cycles
        self.needle_index = random.randrange(self.input_size)
        
    def more(self):
        return self.remaining_cycles > 0
    
    def sense(self):
        haystack = BitString.random(self.input_size)
        self.needle_value = haystack[self.needle_index]
        return haystack
    
    def execute(self, action):
        self.remaining_cycles -= 1
        return action == self.needle_value

We can now instantiate our scenario, as well as the xcs algorithm.

In [10]:
scenario = HaystackScenario(training_cycles=1000, input_size=100)

algorithm = xcs.XCSAlgorithm()

Run the algorithm for different parameter settings, starting from the default ones, and inspect the rules found by the algorithm. Are they consistent with what you expected?

In [11]:
algorithm.exploration_probability = .1
algorithm.ga_threshold = 1
algorithm.crossover_probability = .5
algorithm.do_action_set_subsumption = True
algorithm.do_ga_subsumption = False
algorithm.wildcard_probability = .998
algorithm.deletion_threshold = 1
algorithm.mutation_probability = .002

In [12]:
model = algorithm.new_model(scenario)
model.run(scenario, learn=True)

In [13]:
print(len(model))

11


In [14]:
for rule in model:
    if rule.fitness > .4:
        print(f"{rule.condition} => {rule.action}\t {rule.fitness}")

#############################################################1###################################### => False	 0.7969437513379902
#############################################################1###################################### => True	 0.9814485958029057
#############################################################0###################################### => False	 0.9990403418193906
